# Recommender system
## Collaborative filtering


In [1]:
# importing the neessary libs
import numpy as np
import pandas as pd
from scipy import sparse

In [2]:
def open_create_list(path, sep, column):
    """Function that opens a file and extract a specific column from it creating a list."""
    var = open(path, 'r', encoding='iso-8859-1')  
    new_list = []
    for l in var:
        l = l.split(sep)
        new_list.append(l[column])
    return new_list

### Creating a DataFrame with the important columns from the files

In [3]:
# Columns from u.data: | user id | item id | rating | timestamp |
userId_data = open_create_list('data/u.data', '\t', 0)
movieId_data = open_create_list('data/u.data', '\t', 1)
rating_data = open_create_list('data/u.data', '\t', 2)

# creating an array with the lists data
array_data = np.array([userId_data, movieId_data, rating_data])

In [7]:
# creating a dataframe with the data
data_df = pd.DataFrame(data = array_data)
data_df

,0,1,2,3,4,5,6,7,8,9,...,99990,99991,99992,99993,99994,99995,99996,99997,99998,99999
0,196,186,22,244,166,298,115,253,305,6,...,806,676,721,913,378,880,716,276,13,12
1,242,302,377,51,346,474,265,465,451,86,...,421,538,262,209,78,476,204,1090,225,203
2,3,3,1,2,1,4,2,5,3,3,...,4,4,3,2,3,3,5,1,2,3


In [8]:
# transposing the DF and setting the proper column name
data_df = data_df.T
data_df.columns = 'userID movieID rating'.split()
data_df.head()

,userID,movieID,rating
0,196,242,3
1,186,302,3
2,22,377,1
3,244,51,2
4,166,346,1


In [10]:
# loading the unique movie information
# u.item first 3 columns: | movie id | movie title | release date |
movieId_uniq = open_create_list('data/u.item', '|', 0)
movieName_uniq = open_create_list('data/u.item', '|', 1)
movieDate_uniq = open_create_list('data/u.item', '|', 2)

array_item = np.array([movieId_uniq, movieName_uniq, movieDate_uniq])

In [12]:
# creating a dataframe with the unique movie information 
item_uniq_df = pd.DataFrame(data= array_item)
item_uniq_df = item_uniq_df.T
item_uniq_df.columns = 'movieID Name Date'.split()
item_uniq_df.head()

,movieID,Name,Date
0,1,Toy Story (1995),01-Jan-1995
1,2,GoldenEye (1995),01-Jan-1995
2,3,Four Rooms (1995),01-Jan-1995
3,4,Get Shorty (1995),01-Jan-1995
4,5,Copycat (1995),01-Jan-1995


## Mergind DataFrames
- Both dataframes `data_df` and `item_uniq_df` have the column movieID
- That means that the `merge` method will use this to combine the proper lines
- So, we actully add two new columns to the `data_df`: movie Name and movie Date
- Thus, the `data_and_item_df` if generated

In [13]:
data_and_item_df = pd.merge(data_df, item_uniq_df)
data_and_item_df.head()

,userID,movieID,rating,Name,Date
0,196,242,3,Kolya (1996),24-Jan-1997
1,63,242,3,Kolya (1996),24-Jan-1997
2,226,242,5,Kolya (1996),24-Jan-1997
3,154,242,3,Kolya (1996),24-Jan-1997
4,306,242,5,Kolya (1996),24-Jan-1997


In [14]:
# checking the shape of the final merged dataframe
data_and_item_df.shape

(100000, 5)

In [16]:
# converting the column "rating" from str ---> int
data_and_item_df['rating'] = pd.to_numeric(data_and_item_df['rating'])

In [17]:
# Checking if there's any NaN
data_and_item_df.isnull().sum()

userID     0
movieID    0
rating     0
Name       0
Date       0
dtype: int64

## Making a matrix of **user x item** (movie)
- Probably not every user has rated evey movie, so we will have some NaN now

In [19]:
matrix_user_item = data_and_item_df.pivot_table(index='userID', columns='Name', values='rating')
matrix_user_item.isnull().sum()

Name
'Til There Was You (1997)                934
1-900 (1994)                             938
101 Dalmatians (1996)                    834
12 Angry Men (1957)                      818
187 (1997)                               902
                                        ... 
Young Guns II (1990)                     899
Young Poisoner's Handbook, The (1995)    902
Zeus and Roxanne (1997)                  937
unknown                                  934
Á köldum klaka (Cold Fever) (1994)       942
Length: 1664, dtype: int64

- With that much NaN values we need to decide a threshold
- The dataset downloaded used as min_ratings = 20 for each user
- So, this would be a fair value to assume here as well
- using `dropna` we pass the ( thresh = 20 , axis = 1 )

In [22]:
matrix_user_item.dropna(thresh=20, axis=1, inplace=True)
matrix_user_item.head()

Name,101 Dalmatians (1996),12 Angry Men (1957),187 (1997),2 Days in the Valley (1996),"20,000 Leagues Under the Sea (1954)",2001: A Space Odyssey (1968),"39 Steps, The (1935)",8 1/2 (1963),Absolute Power (1997),"Abyss, The (1989)",...,Wishmaster (1997),With Honors (1994),"Wizard of Oz, The (1939)",Wolf (1994),"Wrong Trousers, The (1993)",Wyatt Earp (1994),Young Frankenstein (1974),Young Guns (1988),Young Guns II (1990),"Young Poisoner's Handbook, The (1995)"
userID,,,,,,,,,,,,,,,,,,,,,
1,2.0,5.0,NaN,NaN,3.0,4.0,NaN,NaN,NaN,3.0,...,NaN,NaN,4.0,NaN,5.0,NaN,5.0,3.0,NaN,NaN
10,NaN,5.0,NaN,NaN,NaN,5.0,4.0,NaN,NaN,4.0,...,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
101,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
102,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN


In [23]:
# checking the new shape of the matrix df
matrix_user_item.shape

(943, 931)

### Now we can replace the NaN == 0 ...


In [ ]:
matrix_user_item = matrix_user_item.fillna(0)

### ... and standardize the values
![image](https://365datascience.com/resources/blog/2018-10-image5-9-1024x591.jpg)

- the ratings are clearly unleveled so we can't compare them before normalizing
- each item has a different mean and number of ratings
- so, if we create a function that standardize the values from each line (user), we will be able to compare them



In [25]:
def standardize(line):
    """Standardize the values from a given line.
    The equation is the Z-score from a normal distribution."""
    new_line = (line - line.mean()) / (line.max() - line.min())
    return new_line

matrix_user_item_std = matrix_user_item.apply(standardize)
matrix_user_item_std

Name,101 Dalmatians (1996),12 Angry Men (1957),187 (1997),2 Days in the Valley (1996),"20,000 Leagues Under the Sea (1954)",2001: A Space Odyssey (1968),"39 Steps, The (1935)",8 1/2 (1963),Absolute Power (1997),"Abyss, The (1989)",...,Wishmaster (1997),With Honors (1994),"Wizard of Oz, The (1939)",Wolf (1994),"Wrong Trousers, The (1993)",Wyatt Earp (1994),Young Frankenstein (1974),Young Guns (1988),Young Guns II (1990),"Young Poisoner's Handbook, The (1995)"
userID,,,,,,,,,,,,,,,,,,,,,
1,-0.227064,0.218667,NaN,NaN,-0.166667,0.007722,NaN,NaN,NaN,-0.147351,...,NaN,NaN,-0.019309,NaN,0.133475,NaN,0.351667,-0.05198,NaN,NaN
10,NaN,0.218667,NaN,NaN,NaN,0.257722,-0.016949,NaN,NaN,0.102649,...,NaN,NaN,0.230691,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
101,0.022936,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
102,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.147351,...,NaN,NaN,NaN,NaN,NaN,NaN,0.018333,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,NaN,0.218667,NaN,NaN,0.166667,-0.242278,NaN,NaN,NaN,NaN,...,NaN,NaN,-0.269309,NaN,NaN,NaN,0.018333,0.19802,-0.193182,NaN
96,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97,NaN,NaN,NaN,NaN,NaN,0.257722,NaN,NaN,NaN,NaN,...,NaN,NaN,0.230691,NaN,0.133475,NaN,0.351667,NaN,NaN,NaN



os usuarios estao disnivelados, nao faz muito sentido compara-los sendo que cada um tem uma média, um numero de avaliacoes diferente...
pra isso, podemos padroniza-los criando uma função